# Decorators and Closures

Function decorators let us "mark" functions in the source code to enhance their behavior in some way.

We'll cover:
- How Python evaluates decorator syntax
- How Python decides whether a variable is local
- Why closures exist and how they work
- What problem is solved by `nonlocal`

We can further tackle:
- Implementing a well-behaved decorator
- Powerful decorators in the standard library: `@cache`, `@lru_cache`, and
`@singledispatch`
- Implementing a parameterized decorator

## Decorators 101

A decorator is a callable that takes another function as an argument (the
decorated function).

A decorator may perform some processing with the decorated function, and
returns it or replaces it with another function or callable object.

In other words, this
```python
@decorate
def target():
    print("running target()")
```
has the same effect as this
```python
def target():
    print("running target()")

target = decorate(target)
```

Example 9-1. A decorator usually replaces a function with a different one

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [2]:
target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

## When Python Executes Decorators

Example 9-2. The `registration.py` module

In [4]:
# tag::REGISTRATION[]

registry = []  # <1>

def register(func):  # <2>
    print(f'running register({func})')  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')


# end::REGISTRATION[]


running register(<function f1 at 0x7fd0d3b43ac0>)
running register(<function f2 at 0x7fd0d3b43be0>)


In [5]:
print('registry ->', registry)

registry -> [<function f1 at 0x7fd0d3b43ac0>, <function f2 at 0x7fd0d3b43be0>]


In [6]:
f1()

running f1()


In [7]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [8]:
f2()

running f2()


In [9]:
f3()

running f3()


`registration.py` holds the above commands in an `if __name__ == '__main__':`
 block. What will be different when running that script directly vs importing?

In [10]:
! python registration.py

running register(<function f1 at 0x7f7ee31d64d0>)
running register(<function f2 at 0x7f7ee31d6560>)
running main()
registry -> [<function f1 at 0x7f7ee31d64d0>, <function f2 at 0x7f7ee31d6560>]
running f1()
running f2()
running f3()


In [11]:
import registration

running register(<function f1 at 0x7fd0d3be8430>)
running register(<function f2 at 0x7fd0d3be83a0>)


In [12]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

## Registration Decorators

Example 9-2 is a little funny in 2 ways:
- The decorator function is defined in the same module as the decorated
function. A real decorator is usually defined in one module and applied to
functions in other modules.
- The `register` decorator returns the same function passed as an argument.
In practice, most decorators define an inner function and return it. These
depend on closures to operate properly.

Let's dig into how python handles closures.

## Variable Scope Rules

Example 9-3. Function reading a local and a global variable

In [13]:
def f1(a):
    print(a)
    print(b)

In [14]:
f1(3)

3


NameError: name 'b' is not defined

In [ ]:
b = 6

f1(3)

Example 9-4. Variable `b` is local, because it is assigned a value inside the
 body of the function

In [15]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [16]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [17]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [18]:
f3(3)

3
6


In [19]:
f3(3)

3
9


In [20]:
if not True:
    variable = 3
else:
    variable = 5


variable

5

There are 2 scopes at play here:
- The module global scope
- The f3 function local scope

Example 9-5. Disassembly of the f1 function from Example 9-3

In [21]:
from dis import dis

dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


Example 9-6. Dissassembly of the f2 function from 9-4

In [22]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## Closures

A closure is a function `f` with an extended scope that encompasses variables
 referenced in the body of `f` that are not global variables or local
 variables of `f`. Such variables must come from the local scope of an outer
 function that encompasses `f`.

Consider the `avg` function to compute the mean of an ever-growing series of
values. Every day a new price is added, and teh average is computed taking
into account all prices so far.

```python
>>> avg(10)
10
>>> avg(11)
10.5
>>> avg(12)
11
```

Example 9-7. `average_oo.py`: a class to calculate a running average

In [23]:
class Averager:

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [24]:
avg = Averager()

In [25]:
avg(10)

10.0

In [26]:
avg(11)

10.5

In [27]:
avg(12)

11.0

Example 9-8. `average.py`: a higher-order function to calculate a running average

In [28]:

def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

Example 9-9. Testing Example 9-8

In [29]:
avg = make_averager()

In [30]:
avg(10)

10.0

In [31]:
avg(11)

10.5

In [32]:
avg(15)

12.0

Where does the `avg` function find the series?

Within `averager`, `series` is a _free variable_. This is a technical term
meaning a variable that is not bound in the local scope.

Example 9-10. Inspecting the function created by `make_averager`

In [33]:
avg.__code__.co_varnames

('new_value', 'total')

In [34]:
avg.__code__.co_freevars

('series',)

In [35]:
avg.series

AttributeError: 'function' object has no attribute 'series'

Example 9-11

In [36]:
avg.__code__.co_freevars

('series',)

In [37]:
avg.__closure__

(<cell at 0x7fd0d3ce3a60: list object at 0x7fd0d41152c0>,)

In [38]:
avg.__closure__[0].cell_contents

[10, 11, 15]

A closure is a function that retains the bindings of the free variables that
exist when teh function is defined, so that they can be used later when the
function is invoked and the defining scope is no longer available.

## The `nonlocal` Declaration

The previous implementation of `make_averager` was not efficient. Instead of
storing the list of values, we could store the total and number of items so far.

Example 9-12. A broken higher-order function to calculate a running average
without keeping all history

In [39]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count = count + 1
        total += new_value
        return total / count

    return averager


In [40]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

To work around this, the `nonlocal` keyword was introduced in python 3.

Example 9-13. Calculate a running average without keeping all history (fixed
with the use of `nonlocal`)

In [41]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [42]:
avg = make_averager()
avg(1)

1.0

In [43]:
avg(2)

1.5

In [44]:
avg(3)

2.0

## Variable Lookup Logic

When a function is defined, the Python bytecode compiler determines how to
fetch a variable `x` that appears in it based on these rules:
- If there is a `global x` decalaration, `x` comes from and is assigned to
the `x` global variable module.
- If there is a `nonlocal x` declaration, `x` comes from and is assigned to
the `x` local variable of the nearest surrounding function where `x` is defined
- If `x` is a parameter or is assigned a value in the function body, then `x`
 is the local variable.
 - If `x` is referenced but is not assigned and is not a parameter:
    - `x` will be looked up in teh local scopes of the surroudning function
  bodies (nonlocal scopes).
    - If not found in surrounding scopes, it will be read from teh module
    global scope
    - If not found in the global scope, it will be read from `__builtins__
    .__dict__`.

In [45]:
x = 5

def function(a, b):
    x = a
    def  inner():
        global x
        nonlocal x
        print(x)

    return inner

SyntaxError: name 'x' is nonlocal and global (3725507329.py, line 6)

## Implementing a Simple Decorator

Example 9-14. `clockdeco0.py`: simple decorator to show the running time of
functions

In [46]:
import time


def clock(func):
    """
    clock decorator
    """
    def clocked(*args):  # <1>
        t0 = time.perf_counter()
        result = func(*args)  # <2>
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked  # <3>

Example 9-15. Using the `clock` decorator

In [47]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)


In [48]:
print("*" * 40, "Calling snoze(.123)")
snooze(.123)

**************************************** Calling snoze(.123)
[0.12604302s] snooze(0.123) -> None


In [49]:
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

In [50]:
print("*" * 40, "Calling factorial(6)")
print(f"6! = {factorial(6)}")

**************************************** Calling factorial(6)
[0.00000053s] factorial(1) -> 1
[0.00001381s] factorial(2) -> 2
[0.00001917s] factorial(3) -> 6
[0.00002352s] factorial(4) -> 24
[0.00002806s] factorial(5) -> 120
[0.00003349s] factorial(6) -> 720
6! = 720


## How It Works

In [51]:
factorial.__name__

'clocked'

This is the typical behavior of a decorator: it replaces the decorated
function with a new function that accepts the same arguments and (usually)
returns whatever the decorated function was supposed to return, while also
doing some extra processing.

The `clock` decorator in 9-14 has a few shortcomings:
- no support for keyword arguments
- it masks the `__name__` and `__doc__` of the decorated function


In [52]:
@clock
def func_with_kwargs(a = 2, b = 3):
    """
    doc of func_with_kwargs
    """
    return a + b


In [53]:
func_with_kwargs(a=2)

TypeError: clock.<locals>.clocked() got an unexpected keyword argument 'a'

In [54]:
func_with_kwargs.__name__

'clocked'

`functools.wraps` fixes both of these.

Example 9 - 16. `clockdeco.py`: an improved clock decorator


In [59]:
import time
import functools


def clock_better(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked


In [60]:
@clock_better
def func_with_kwargs(a = 2, b = 3):
    """
    doc of func_with_kwargs
    """
    return a + b

In [61]:
func_with_kwargs(a=2)

[0.00000148s] func_with_kwargs(a=2) -> 5


5

In [62]:
func_with_kwargs.__name__

'clocked'

# Decorators in the Standard Library

There are 3
- `property`
- `classmethod`
- `staticmethod`

There are more in the `functools` module, namely
- `cache`
- `lru_cache`
- `singledispatch`

## Memoization with functools.cache

Memoization is an optimization technique that works by saving the results of
previous invocations of an expensive function, avoiding repeat computations
on previously used arguments.

Example 9-17. The very costly recursive way to compute the nth number in the
Fibonacci series

In [63]:
@clock_better
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [64]:
fibonacci(6)

[0.00000050s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00024788s] fibonacci(2) -> 1
[0.00000032s] fibonacci(1) -> 1
[0.00000044s] fibonacci(0) -> 0
[0.00000031s] fibonacci(1) -> 1
[0.00001190s] fibonacci(2) -> 1
[0.00002334s] fibonacci(3) -> 2
[0.00028380s] fibonacci(4) -> 3
[0.00000028s] fibonacci(1) -> 1
[0.00000027s] fibonacci(0) -> 0
[0.00000027s] fibonacci(1) -> 1
[0.00001024s] fibonacci(2) -> 1
[0.00002072s] fibonacci(3) -> 2
[0.00000025s] fibonacci(0) -> 0
[0.00000032s] fibonacci(1) -> 1
[0.00001036s] fibonacci(2) -> 1
[0.00000026s] fibonacci(1) -> 1
[0.00000045s] fibonacci(0) -> 0
[0.00000026s] fibonacci(1) -> 1
[0.00001023s] fibonacci(2) -> 1
[0.00002038s] fibonacci(3) -> 2
[0.00004092s] fibonacci(4) -> 3
[0.00007166s] fibonacci(5) -> 5
[0.00036685s] fibonacci(6) -> 8


8

Example 9-18. Faster implementation using caching

In [65]:
import functools

@functools.cache
@clock_better
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)


In [66]:
fibonacci(6)

[0.00000047s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00005119s] fibonacci(2) -> 1
[0.00000057s] fibonacci(3) -> 2
[0.00006238s] fibonacci(4) -> 3
[0.00000045s] fibonacci(5) -> 5
[0.00007295s] fibonacci(6) -> 8


8

## Stacked Decorators

```python
@alpha
@beta
def my_fn():
    ...
```

is the same as

```python
my_fn = alpha(beta(my_fn))
```

`functools.cache` can consume all available memory if there is a large number
 of cache entries.

It's fine for short, command-line scripts.

In long-running processes, `functools.lru_cache` with a suitable `maxsize`
parameter is more suitable.

## Using `lru_cache`

lru = least recently used. Older entries that have not been read for a while
are discarded.

In [ ]:
from functools import lru_cache

@lru_cache
def costly_function(a, b):
    ...

In [ ]:
@lru_cache()
def costly_function(a, b):
    ...

In the two above cases, the default parameters are used.

In [ ]:
@lru_cache(maxsize=2**20, typed=True)
def costly_function(a, b):
    ...

## Single Dispatch Generic Functions

Imagine we are creating a tool to debug web applications. We want to generate
 HTML displays for different types of Python objects.

```python
>>> htmlize({1, 2, 3})  # <1>
'<pre>{1, 2, 3}</pre>'
>>> htmlize(abs)
'<pre>&lt;built-in function abs&gt;</pre>'
>>> htmlize('Heimlich & Co.\n- a game')  # <2>
'<p>Heimlich &amp; Co.<br/>\n- a game</p>'
>>> htmlize(42)  # <3>
'<pre>42 (0x2a)</pre>'
>>> print(htmlize(['alpha', 66, {3, 2, 1}]))  # <4>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>
>>> htmlize(True)  # <5>
'<pre>True</pre>'
>>> htmlize(fractions.Fraction(2, 3))  # <6>
'<pre>2/3</pre>'
>>> htmlize(2/3)   # <7>
'<pre>0.6666666666666666 (2/3)</pre>'
>>> htmlize(decimal.Decimal('0.02380952'))
'<pre>0.02380952 (1/42)</pre>'
```

The java way to handle this would be multiple function definitions for
`htmlize()`, each accepting a different type. But python does not support
method overloading.

A solution: turn `htmlize` into a dispatch function, with a chain of
`if/elif` calling specialized functions like `htmlize_str`, `htmlize_int`,
etc. But that's not extensible.

The `functools.singledispatch` decorator allows for different modules to
contribute to the overall solution.

Example 9-20. `@singledispatch` creates a custom `@htmlize.register` to
bundle several functions into a generic function

In [78]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers


@singledispatch  # <1>
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register  # <2>
def _(text: str) -> str:  # <3>
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register  # <4>
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register  # <5>
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register  # <6>
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)  # <7>
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)  # <8>
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'


In [68]:
htmlize("hello")

'<p>hello</p>'

In [69]:
htmlize([1,2,3])

'<ul>\n<li><pre>1 (0x1)</pre></li>\n<li><pre>2 (0x2)</pre></li>\n<li><pre>3 (0x3)</pre></li>\n</ul>'

In [70]:
htmlize(7)

'<pre>7 (0x7)</pre>'

In [71]:
htmlize( 1/2 )

'<pre>0.5 (1/2)</pre>'

In [72]:
htmlize(.5)

'<pre>0.5 (1/2)</pre>'

## Parameterized Decorators

Example 9-21. Abriged `registration.py` module from Example 9-2, repeated
here for convenience


In [79]:
registry = []

def register(func):
    print(f"running register({func}")
    registry.append(func)
    return func


@register
def f1():
    print('running f1()')

running register(<function f1 at 0x7fd0d3b43a30>


In [80]:
print('registry ->', registry)

registry -> [<function f1 at 0x7fd0d3b43a30>]


### A parameterized Registration Decorator

Example 9-22. To accept parameters, the new `register` decorator must be
called as a function

In [81]:
registry = set()  # <1>

def register(active=True):  # <2>
    def decorate(func):  # <3>
        print('running register'
              f'(active={active})->decorate({func})')
        if active:   # <4>
            registry.add(func)
        else:
            registry.discard(func)  # <5>

        return func  # <6>
    return decorate  # <7>

@register(active=False)  # <8>
def f1():
    print('running f1()')

@register()  # <9>
def f2():
    print('running f2()')

def f3():
    print('running f3()')


running register(active=False)->decorate(<function f1 at 0x7fd0d4399b40>)
running register(active=True)->decorate(<function f2 at 0x7fd0d4399ea0>)


In [82]:
registry

{<function __main__.f2()>}

Example 9-23. Using the registration_param module

In [83]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x7fd0d415d750>)


<function __main__.f3()>

In [84]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [85]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x7fd0d4399ea0>)


<function __main__.f2()>

In [86]:
registry

{<function __main__.f3()>}

## The Parameterized Clock Decorator

Example 9-24. Module `clockdeco_param.py`: the parameterized clock decorator

In [87]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):  # <1>
    def decorate(func):      # <2>
        def clocked(*_args): # <3>
            t0 = time.perf_counter()
            _result = func(*_args)  # <4>
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)  # <5>
            result = repr(_result)  # <6>
            print(fmt.format(**locals()))  # <7>
            return _result  # <8>
        return clocked  # <9>
    return decorate  # <10>


In [88]:
@clock()  # <11>
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12672099s] snooze(0.123) -> None
[0.12593999s] snooze(0.123) -> None
[0.12715239s] snooze(0.123) -> None


Example 9-25. `clockdeco_param_demo1.py`

In [89]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12801719899653108s
snooze: 0.12702987400189159s
snooze: 0.1257736420011497s


Example 9-26. `clockdeco_param_demo2.py`

In [90]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.128s
snooze(0.123) dt=0.128s
snooze(0.123) dt=0.127s


## A Class-Based Clock Decorator

Example 9-27. Module `clockdeco_cls.py`: parameterized clock decorator
implemented as a class

In [91]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock:  # <1>

    def __init__(self, fmt=DEFAULT_FMT):  # <2>
        self.fmt = fmt

    def __call__(self, func):  # <3>
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)  # <4>
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked
# end::CLOCKDECO_CLS[]


In [92]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12748116s] snooze(0.123) -> None
[0.12777819s] snooze(0.123) -> None
[0.12766780s] snooze(0.123) -> None
